In [1]:
# ==============================================
# Cross-Attention Expansion of GRU + MLP Embeddings
# ==============================================

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model

In [2]:
# ---------------------------
# Load embeddings and schedule
# ---------------------------
gru_df = pd.read_csv('gru_team_embeddings_by_week.csv')   # columns: team, seasonweek, emb_0..emb_15
mlp_df = pd.read_csv('mlp_2023_2024_embeddings.csv')      # columns: team, mlp_emb_0..mlp_emb_11
sched  = pd.read_excel('nfl_2024_schedule.xlsx')          # columns: Seasonweek, Away Team, Home Team, Winner

gru_cols = [c for c in gru_df.columns if c.startswith('emb_')]
mlp_cols = [c for c in mlp_df.columns if c.startswith('mlp_emb_')]

team_to_mlp = mlp_df.set_index('team')[mlp_cols].astype(np.float32)

In [3]:
# ---------------------------
# Helper: team-week vector
# ---------------------------
def team_week_vector(team, week):
    row = gru_df[(gru_df['team']==team) & (gru_df['seasonweek']==week)]
    if row.empty:
        raise ValueError(f"No GRU embedding for {team} at {week}")
    gru_vec = row[gru_cols].values.astype(np.float32).reshape(-1)
    mlp_vec = team_to_mlp.loc[team].values.astype(np.float32).reshape(-1)
    return np.concatenate([gru_vec, mlp_vec], axis=0)  # (28,)

In [4]:
# ---------------------------
# Build away/home matrices
# ---------------------------
games = []
for _, r in sched.iterrows():
    week = int(r['Seasonweek'])
    away = r['Away Team']
    home = r['Home Team']
    try:
        a_vec = team_week_vector(away, week)
        h_vec = team_week_vector(home, week)
        games.append({
            'week': week, 'away': away, 'home': home, 'winner': r.get('Winner'),
            'away_vec': a_vec, 'home_vec': h_vec
        })
    except Exception as e:
        print(f"❌ Skip {away}@{home} wk {week}: {e}")

A = np.stack([g['away_vec'] for g in games], axis=0)  # (N, 28)
B = np.stack([g['home_vec'] for g in games], axis=0)  # (N, 28)

In [5]:
# ---------------------------
# Cross-Attention Encoder (concat expansion)
# ---------------------------
def build_cross_attention_encoder(d_team=28, token_dim=4, n_heads=2, ff_dim=32, dropout=0.1):
    n_tokens = d_team // token_dim
    d_model = token_dim * 2

    away_in = layers.Input(shape=(d_team,), name='away_vec')
    home_in = layers.Input(shape=(d_team,), name='home_vec')

    a_tok = layers.Reshape((n_tokens, token_dim))(away_in)
    h_tok = layers.Reshape((n_tokens, token_dim))(home_in)

    a_proj = layers.Dense(d_model)(a_tok)
    h_proj = layers.Dense(d_model)(h_tok)

    mha = layers.MultiHeadAttention(num_heads=n_heads, key_dim=d_model//n_heads, dropout=dropout)
    a_cross = mha(query=a_proj, value=h_proj, key=h_proj)
    h_cross = mha(query=h_proj, value=a_proj, key=a_proj)

    a_cross = layers.LayerNormalization()(a_proj + a_cross)
    h_cross = layers.LayerNormalization()(h_proj + h_cross)

    ff = tf.keras.Sequential([layers.Dense(ff_dim, activation='relu'),
                              layers.Dense(d_model)])
    a_ff = layers.LayerNormalization()(a_cross + ff(a_cross))
    h_ff = layers.LayerNormalization()(h_cross + ff(h_cross))

    a_pooled = layers.GlobalAveragePooling1D()(a_ff)
    h_pooled = layers.GlobalAveragePooling1D()(h_ff)

    a_att = layers.Dense(d_team)(a_pooled)
    h_att = layers.Dense(d_team)(h_pooled)

    # Concatenate: original + attended
    a_out = layers.Concatenate()([away_in, a_att])  # (batch, 56)
    h_out = layers.Concatenate()([home_in, h_att])  # (batch, 56)

    return Model([away_in, home_in], [a_out, h_out], name="cross_attn_concat")

xatt_model = build_cross_attention_encoder(d_team=28)
A_exp, B_exp = xatt_model.predict([A, B], verbose=0)

X = np.concatenate([A_exp, B_exp], axis=1)  # (N, 112)

In [6]:
# ---------------------------
# Build DataFrame with names
# ---------------------------
gru_labels = [f"GRU_emb_{i}" for i in range(16)]
mlp_labels = [f"MLP_feat_{i}" for i in range(12)]

feature_names = (
    [f"A_ORIG_{c}" for c in gru_labels + mlp_labels] +
    [f"A_ATT_{c}"  for c in gru_labels + mlp_labels] +
    [f"B_ORIG_{c}" for c in gru_labels + mlp_labels] +
    [f"B_ATT_{c}"  for c in gru_labels + mlp_labels]
)

rows = []
for i, g in enumerate(games):
    row = {
        'week': g['week'],
        'away': g['away'],
        'home': g['home'],
        'label': 1 if g.get('winner') == g['away'] else 0
    }
    for j, val in enumerate(X[i]):
        row[feature_names[j]] = float(val)
    rows.append(row)

xatt_df = pd.DataFrame(rows)
xatt_df.to_csv("cross_attention_game_features.csv", index=False)

print("✅ Saved cross-attention features to cross_attention_game_features.csv")
xatt_df.head()

✅ Saved cross-attention features to cross_attention_game_features.csv


,week,away,home,label,A_ORIG_GRU_emb_0,A_ORIG_GRU_emb_1,A_ORIG_GRU_emb_2,A_ORIG_GRU_emb_3,A_ORIG_GRU_emb_4,A_ORIG_GRU_emb_5,...,B_ATT_MLP_feat_2,B_ATT_MLP_feat_3,B_ATT_MLP_feat_4,B_ATT_MLP_feat_5,B_ATT_MLP_feat_6,B_ATT_MLP_feat_7,B_ATT_MLP_feat_8,B_ATT_MLP_feat_9,B_ATT_MLP_feat_10,B_ATT_MLP_feat_11
0,202401,RAV,KAN,0,0.000000,0.157643,0.000000,0.039637,0.0,0.0,...,0.036323,-0.103077,0.492892,-0.310335,0.367838,-1.018103,-0.826827,-0.146097,-0.378594,-0.426404
1,202401,GNB,PHI,0,0.000000,0.017632,0.000000,0.165190,0.0,0.0,...,-0.200059,0.110334,-0.316570,0.400149,0.147962,-0.083243,0.065248,-0.249367,0.247251,-0.373150
2,202401,PIT,ATL,1,0.000000,0.000000,0.082032,0.000000,0.0,0.0,...,0.191151,-0.208099,0.424873,0.153504,0.660798,-0.772985,-0.679726,-0.033389,-0.225909,-0.662226
3,202401,CRD,BUF,0,0.114795,0.190519,0.000000,0.018953,0.0,0.0,...,0.424147,-0.241486,-0.806257,0.101240,0.238124,0.141799,-0.240272,-0.233433,-0.105100,-0.129934
4,202401,CAR,NOR,0,0.527900,0.218061,0.000000,0.193355,0.0,0.0,...,0.289596,0.175733,-0.097946,0.347338,0.270809,-0.724981,-0.367613,-0.279371,-0.066018,-0.337511
